In [1]:
import numpy as np
import torch

In [2]:
# Example: define the radial grid (size 140)
r = np.random.rand(140)  # Example radial grid (adjust to your case)

# Assuming 3D arrays for rho, v_r (shape: 140, 110, 128)
rho = np.random.rand(140, 110, 128)  # Example 3D density array
v_r = np.random.rand(140, 110, 128)  # Example 3D radial velocity array

g = 10  # Example gravitational acceleration array (1D, size 140)
p = np.random.rand(140, 110, 128)  # Example pressure array (1D, size 140)

In [3]:
# Broadcast 1D arrays to the shape (140, 110, 128)
r_expanded = np.broadcast_to(r[:, np.newaxis, np.newaxis], (140, 110, 128))  # Broadcasting r

In [4]:
# Compute first derivatives using np.gradient for the radial direction (axis 0 is the radial direction)
d_rho_dr = np.gradient(rho, r, axis=0)  # First derivative of density w.r.t. r (along axis 0)
dv_r_dr = np.gradient(v_r, r, axis=0)  # First derivative of v_r (radial velocity) w.r.t. r (along axis 0)
d_p_dr = np.gradient(p, r, axis=0)  # First derivative of pressure w.r.t. r (1D array)

In [5]:
# Terms in the equation:
term1 = rho * v_r * dv_r_dr  # Convective term: rho * v_r * (dv_r / dr)
term2 = -d_p_dr  # Pressure gradient term: - dp / dr
term3 = rho * g  # Gravitational term: rho * g_r (Broadcast g_r to match rho's shape)

# print(term1.shape, term2.shape, term3.shape, term4.shape, term5.shape, term6.shape, term7.shape)

# Final equation (note the result will have the same shape as rho, v_r, p, etc.)
equation_result = -term1 + term2 + term3

# The result should be the value on the left-hand side of the equation:
print(equation_result.shape)  # Verify the shape of the result (should be the same as the input arrays)


(140, 110, 128)


In [6]:
equation_result[0, :5, :5]

array([[ 2.75432901,  3.76756449,  1.08817247,  1.32738702,  4.18309227],
       [ 8.8459812 ,  5.91843877,  4.21359835,  6.8749336 , 10.02880663],
       [ 1.74740802,  1.94056367,  1.09248574,  4.62110055, -0.09307564],
       [ 0.89373813,  3.40525045,  2.0516053 ,  4.28251988,  8.66245548],
       [-0.65022464,  5.33669415,  5.95970196,  4.18587231, -0.44483   ]])

In [7]:
rho_tensor = torch.tensor(rho, requires_grad=True)
v_r_tensor = torch.tensor(v_r, requires_grad=True)
p_tensor = torch.tensor(p, requires_grad=True)
r_tensor = torch.tensor(r, requires_grad=True)

In [8]:
dr = float(r_tensor[1]-r_tensor[0])

In [9]:
# Compute first derivatives using np.gradient for the radial direction (axis 0 is the radial direction)
d_rho_dr = torch.gradient(rho_tensor, axis=0)[0] / dr  # First derivative of density w.r.t. r (along axis 0)
dv_r_dr = torch.gradient(v_r_tensor, axis=0)[0] / dr    # First derivative of v_r (radial velocity) w.r.t. r (along axis 0)
d_p_dr = torch.gradient(p_tensor, axis=0)[0]  / dr   # First derivative of pressure w.r.t. r (1D array)

In [10]:
# Terms in the equation:
term1 = rho_tensor * v_r_tensor * dv_r_dr  # Convective term: rho * v_r * (dv_r / dr)
term2 = -d_p_dr  # Pressure gradient term: - dp / dr
term3 = rho_tensor * g  # Gravitational term: rho * g_r (Broadcast g_r to match rho's shape)

# print(term1.shape, term2.shape, term3.shape, term4.shape, term5.shape, term6.shape, term7.shape)

# Final equation (note the result will have the same shape as rho, v_r, p, etc.)
equation_result = -term1 + term2 + term3

# The result should be the value on the left-hand side of the equation:
print(equation_result.shape)  # Verify the shape of the result (should be the same as the input arrays)


torch.Size([140, 110, 128])


In [11]:
equation_result[0, :5, :5]

tensor([[ 2.7543,  3.7676,  1.0882,  1.3274,  4.1831],
        [ 8.8460,  5.9184,  4.2136,  6.8749, 10.0288],
        [ 1.7474,  1.9406,  1.0925,  4.6211, -0.0931],
        [ 0.8937,  3.4053,  2.0516,  4.2825,  8.6625],
        [-0.6502,  5.3367,  5.9597,  4.1859, -0.4448]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)

In [17]:
dv_r_dr = (v_r_tensor[1:, :, :] - v_r_tensor[:-1, :, :]) / (r_tensor[1] - r_tensor[0])  # First derivative of v_r (along radial axis)

# Compute the gradient of p with respect to r (pressure gradient)
dp_dr = (p_tensor[1:] - p_tensor[:-1]) / (r_tensor[1] - r_tensor[0])

term1 = rho_tensor * v_r_tensor * dv_r_dr  # Convective term: rho * v_r * (dv_r / dr)
term2 = -dp_dr  # Pressure gradient term: - dp / dr
term3 = rho_tensor * g  # Gravitational term: rho * g

value = term2 + term3 - term1
value[0, :5, :5]

RuntimeError: The size of tensor a (140) must match the size of tensor b (139) at non-singleton dimension 0

In [12]:
v_pred_flattened = v_r_tensor.view(-1)  # Flatten for gradient computation
r_flattened = r_tensor.repeat(
    v_pred_flattened.shape[0] // len(r_tensor)
)  # Repeat r to match the flattened v_pred shape

# Calculate the gradient of v_pred with respect to r using autograd
dv_r_dr = torch.autograd.grad(
    v_pred_flattened,
    r_flattened,
    grad_outputs=torch.ones_like(v_pred_flattened),
    create_graph=True,
    allow_unused=True
)[
    0
]  # First derivative of v_r
print(dv_r_dr)

# Similarly, compute the gradient of p (pressure) with respect to r
p_flattened = p_tensor.view(-1)  # Flatten p for gradient computation
dp_dr = torch.autograd.grad(
    p_flattened,
    r_flattened,
    grad_outputs=torch.ones_like(p_flattened),
    create_graph=True,
    allow_unused=True
)[
    0
]  # First derivative of pressure
print(dp_dr)

term1 = rho_tensor * v_r_tensor * dv_r_dr.view(rho_tensor.shape)  # Convective term: rho * v_r * (dv_r / dr)
term2 = -dp_dr.view(rho_tensor.shape)  # Pressure gradient term: - dp / dr
term3 = rho_tensor * g  # Gravitational term: rho * g

value = term2 + term3 - term1

None
None


AttributeError: 'NoneType' object has no attribute 'view'